## Linear compartment models with constant input

We consider three series of linear compartment models: cyclic, catenary, and mamillary. For each of the models, we replace each reaction rate $a_{ij}$ with $b_{ij} + c_{ij}x_0$, where $b_{ij}$ and $c_{ij}$ are scalar parameters, and $x_0$ is a constant input. We report the results and runtimes.

In [1]:
using BenchmarkTools
using DataFrames
include("../experiments_bounds.jl")
include("linear_compartment.jl");

$\require{action}$

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2020 by The Oscar Development Team


In [2]:
function run_model(func, min_n, max_n)
    """
    func is a function for generating a model:
    cycle, catenary, mammilary
    min_n and max_n define the range of sizes to test
    """
    results = []
    runtimes = []
    for n in min_n:max_n
        ode = linear_compartment_model(func(n), [1])
        x1 = gens(ode.poly_ring)[2]
        time = @belapsed bound_number_experiments($ode, [$x1])
        bnd = bound_number_experiments(ode, [x1])
        push!(results, bnd)
        push!(runtimes, time)
    end
    return df = DataFrame(n = min_n:max_n, results = results, runtimes = runtimes)
end;

In [3]:
run_model(cycle, 3, 15)

,n,results,runtimes
,Int64,Any,Any
1,3,"(loc = 3, glob = 4)",0.244285
2,4,"(loc = 3, glob = 4)",0.432294
3,5,"(loc = 3, glob = 4)",0.886086
4,6,"(loc = 3, glob = 4)",1.53951
5,7,"(loc = 3, glob = 4)",2.38671
6,8,"(loc = 3, glob = 4)",3.61151
7,9,"(loc = 3, glob = 4)",5.07298
8,10,"(loc = 3, glob = 4)",9.25867
9,11,"(loc = 3, glob = 4)",12.4893


In [4]:
run_model(catenary, 3, 15)

,n,results,runtimes
,Int64,Any,Any
1,3,"(loc = 4, glob = 5)",0.461768
2,4,"(loc = 5, glob = 6)",1.96651
3,5,"(loc = 5, glob = 6)",3.78969
4,6,"(loc = 5, glob = 6)",7.00651
5,7,"(loc = 5, glob = 6)",13.2395
6,8,"(loc = 5, glob = 6)",21.0293
7,9,"(loc = 5, glob = 6)",29.9834
8,10,"(loc = 5, glob = 6)",43.0187
9,11,"(loc = 5, glob = 6)",64.1456


In [5]:
run_model(mammilary, 3, 15)

,n,results,runtimes
,Int64,Any,Any
1,3,"(loc = 4, glob = 5)",0.46369
2,4,"(loc = 5, glob = 6)",1.91757
3,5,"(loc = 5, glob = 6)",3.72879
4,6,"(loc = 5, glob = 6)",6.94631
5,7,"(loc = 5, glob = 6)",13.2
6,8,"(loc = 5, glob = 6)",21.1801
7,9,"(loc = 5, glob = 6)",30.1944
8,10,"(loc = 5, glob = 6)",43.2578
9,11,"(loc = 5, glob = 6)",64.1186
